In [1]:
# !pip install episcanpy
# !pip install scanpy
# !pip install anndata
# !pip install desc
# !pip install display
# !pip install --pre deepchem
# !pip install nfp

In [3]:
import numpy as np
import pandas as pd
import scanpy as sc
import gzip
import csv
import os
import desc as desc
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import RDKFingerprint
from sklearn.metrics import classification_report, plot_confusion_matrix, confusion_matrix
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.21.6 scipy==1.4.1 pandas==1.3.5 scikit-learn==1.0.2 statsmodels==0.10.2 python-igraph==0.9.10 louvain==0.7.1 pynndescent==0.5.6


In [4]:
from google.colab import drive
drive.mount('/content/drive')
# drive.mount('/content/drive/My Drive/Projects and research stuffs')
!ls "/content/drive/My Drive/Projects and research stuffs/Drug Synergy"

Mounted at /content/drive
'Dataset for GNN'  'Drug Synergy Prediction'


## What is a Fingerprint?

Deep learning models almost always take arrays of numbers as their inputs. If we want to process molecules with them, we somehow need to represent each molecule as one or more arrays of numbers.

Many (but not all) types of models require their inputs to have a fixed size. This can be a challenge for molecules, since different molecules have different numbers of atoms. If we want to use these types of models, we somehow need to represent variable sized molecules with fixed sized arrays.

Fingerprints are designed to address these problems. A fingerprint is a fixed length array, where different elements indicate the presence of different features in the molecule. If two molecules have similar fingerprints, that indicates they contain many of the same features, and therefore will likely have similar chemistry.

DeepChem supports a particular type of fingerprint called an "Extended Connectivity Fingerprint", or "ECFP" for short. They also are sometimes called "circular fingerprints". The ECFP algorithm begins by classifying atoms based only on their direct properties and bonds. Each unique pattern is a feature. For example, "carbon atom bonded to two hydrogens and two heavy atoms" would be a feature, and a particular element of the fingerprint is set to 1 for any molecule that contains that feature. It then iteratively identifies new features by looking at larger circular neighborhoods. One specific feature bonded to two other specific features becomes a higher level feature, and the corresponding element is set for any molecule that contains it. This continues for a fixed number of iterations, most often two.

In [5]:
import deepchem as dc
dc.__version__

'2.6.1'

#### Creating Circular Fingerprints ( Neural based fingerprint creation - GNN )

In [6]:
train_data_path = "/content/drive/My Drive/Projects and research stuffs/Drug Synergy/Dataset for GNN"
main_data = pd.read_csv(train_data_path+'/final_data.csv')
print(main_data.shape)
main_data.head()


(23052, 8)


,Unnamed: 0,drug_a_name,drug_b_name,cell_line,synergy,fold,drug_a_structure,drug_b_structure
0,5-FU_ABT-888_A2058,5-FU,ABT-888,A2058,7.693530,2,O=c1[nH]cc(F)c(=O)[nH]1,CC1(c2nc3c(C(N)=O)cccc3[nH]2)CCCN1
1,5-FU_ABT-888_A2780,5-FU,ABT-888,A2780,7.778053,2,O=c1[nH]cc(F)c(=O)[nH]1,CC1(c2nc3c(C(N)=O)cccc3[nH]2)CCCN1
2,5-FU_ABT-888_A375,5-FU,ABT-888,A375,-1.198505,2,O=c1[nH]cc(F)c(=O)[nH]1,CC1(c2nc3c(C(N)=O)cccc3[nH]2)CCCN1
3,5-FU_ABT-888_A427,5-FU,ABT-888,A427,2.595684,2,O=c1[nH]cc(F)c(=O)[nH]1,CC1(c2nc3c(C(N)=O)cccc3[nH]2)CCCN1
4,5-FU_ABT-888_CAOV3,5-FU,ABT-888,CAOV3,-5.139971,2,O=c1[nH]cc(F)c(=O)[nH]1,CC1(c2nc3c(C(N)=O)cccc3[nH]2)CCCN1


In [7]:
from sklearn.preprocessing import StandardScaler
data_for_gnn = main_data[['drug_a_structure', 'drug_b_structure', 'cell_line', 'synergy']].copy()
data_for_gnn = data_for_gnn.sample(frac=1).reset_index().drop(['index'], 1)

scaler = StandardScaler().fit(data_for_gnn['synergy'].values.reshape(-1,1))
scaled_ops = scaler.transform(data_for_gnn['synergy'].values.reshape(-1,1))
data_for_gnn['synergy'] = scaled_ops
data_for_gnn.head()

,drug_a_structure,drug_b_structure,cell_line,synergy
0,CN1C(=O)C=CC2(C)C3CCC4(C)C(NC(=O)OCC(F)(F)F)CC...,N.N.O=C(O)C1(C(=O)O)CCC1.[Pt],A2780,0.121266
1,CC1(c2nc3c(C(N)=O)cccc3[nH]2)CCCN1,COC1=C2CC(C)CC(OC)C(O)C(C)C=C(C)C(OC(N)=O)C(OC...,SKOV3,-0.694117
2,CN1C(=O)C=CC2(C)C3CCC4(C)C(NC(=O)OCC(F)(F)F)CC...,CCc1c2c(nc3ccc(O)cc13)-c1cc3c(c(=O)n1C2)COC(=O...,UWB1289,-0.088576
3,COc1cccc2c1C(=O)c1c(O)c3c(c(O)c1C2=O)CC(O)(C(=...,COC1CC2CCC(C)C(O)(O2)C(=O)C(=O)N2CCCCC2C(=O)OC...,NCIH460,0.277679
4,CCC1(O)C(=O)OCc2c1cc1n(c2=O)Cc2cc3c(CN(C)C)c(O...,Cn1c(=O)n(-c2ccc(C(C)(C)C#N)cc2)c2c3cc(-c4cnc5...,PA1,-0.649434


In [8]:
data_for_gnn.shape

(23052, 4)

In [8]:
#### Need to remove all drug smiles with '.' in them - this '.' creates separate isolated and disconnected graphs and it gets hard to calculate adjacency matrix and update them

In [ ]:
from collections import defaultdict

import numpy as np

from rdkit import Chem

import torch


def create_atoms(mol, atom_dict):
    """Transform the atom types in a molecule (e.g., H, C, and O)
    into the indices (e.g., H=0, C=1, and O=2).
    Note that each atom index considers the aromaticity.
    """
    atoms = [a.GetSymbol() for a in mol.GetAtoms()]
    for a in mol.GetAromaticAtoms():
        i = a.GetIdx()
        atoms[i] = (atoms[i], 'aromatic')
    atoms = [atom_dict[a] for a in atoms]
    return np.array(atoms)


def create_ijbonddict(mol, bond_dict):
    """Create a dictionary, in which each key is a node ID
    and each value is the tuples of its neighboring node
    and chemical bond (e.g., single and double) IDs.
    """
    i_jbond_dict = defaultdict(lambda: [])
    for b in mol.GetBonds():
        i, j = b.GetBeginAtomIdx(), b.GetEndAtomIdx()
        bond = bond_dict[str(b.GetBondType())]
        i_jbond_dict[i].append((j, bond))
        i_jbond_dict[j].append((i, bond))
    return i_jbond_dict


def extract_fingerprints(radius, atoms, i_jbond_dict,
                         fingerprint_dict, edge_dict):
    """Extract the fingerprints from a molecular graph
    based on Weisfeiler-Lehman algorithm.
    """

    if (len(atoms) == 1) or (radius == 0):
        nodes = [fingerprint_dict[a] for a in atoms]

    else:
        nodes = atoms
        i_jedge_dict = i_jbond_dict

        for _ in range(radius):

            """Update each node ID considering its neighboring nodes and edges.
            The updated node IDs are the fingerprint IDs.
            """
            nodes_ = []
            for i, j_edge in i_jedge_dict.items():
                neighbors = [(nodes[j], edge) for j, edge in j_edge]
                fingerprint = (nodes[i], tuple(sorted(neighbors)))
                nodes_.append(fingerprint_dict[fingerprint])

            """Also update each edge ID considering
            its two nodes on both sides.
            """
            i_jedge_dict_ = defaultdict(lambda: [])
            for i, j_edge in i_jedge_dict.items():
                for j, edge in j_edge:
                    both_side = tuple(sorted((nodes[i], nodes[j])))
                    edge = edge_dict[(both_side, edge)]
                    i_jedge_dict_[i].append((j, edge))

            nodes = nodes_
            i_jedge_dict = i_jedge_dict_

    return np.array(nodes)


def split_dataset(dataset, ratio):
    """Shuffle and split a dataset."""
    np.random.seed(1234)  # fix the seed for shuffle.
    np.random.shuffle(dataset)
    n = int(ratio * len(dataset))
    return dataset[:n], dataset[n:]


def create_datasets(task, data_for_gnn, radius, device):

    dir_dataset =  ''
    print(type(data_for_gnn))
    print(data_for_gnn.shape)
    """Initialize x_dict, in which each key is a symbol type
    (e.g., atom and chemical bond) and each value is its index.
    """
    atom_dict = defaultdict(lambda: len(atom_dict))
    bond_dict = defaultdict(lambda: len(bond_dict))
    fingerprint_dict = defaultdict(lambda: len(fingerprint_dict))
    edge_dict = defaultdict(lambda: len(edge_dict))
   
    
    non_working_smiles_index = []
    def create_dataset(data):

#         print(filename)

#         """Load a dataset."""
#         with open(dir_dataset + filename, 'r') as f:
#             smiles_property = f.readline().strip().split()
#             data_original = f.read().strip().split('\n')

#         """Exclude the data contains '.' in its smiles."""
#         data_original = [data for data in data_original
#                          if '.' not in data.split()[0]]

        dataset = []

        for data_index in range(len(data)):
#             print(data_index)
            try:
        #             smiles, property = data.strip().split()
                smiles1, smiles2, property = data.iloc[data_index]['drug_a_structure'], data.iloc[data_index]['drug_b_structure'], data.iloc[data_index]['synergy']

                """Create each data with the above defined functions."""
                # for drug A:
                mol = Chem.AddHs(Chem.MolFromSmiles(smiles1))
                atoms = create_atoms(mol, atom_dict)
                molecular_size1 = len(atoms)
                i_jbond_dict = create_ijbonddict(mol, bond_dict)
                fingerprints1 = extract_fingerprints(radius, atoms, i_jbond_dict,
                                                    fingerprint_dict, edge_dict)
                adjacency1 = Chem.GetAdjacencyMatrix(mol)

                # for drug B:
                mol = Chem.AddHs(Chem.MolFromSmiles(smiles2))
                atoms = create_atoms(mol, atom_dict)
                molecular_size2 = len(atoms)
                i_jbond_dict = create_ijbonddict(mol, bond_dict)
                fingerprints2 = extract_fingerprints(radius, atoms, i_jbond_dict,
                                                    fingerprint_dict, edge_dict)
                adjacency2 = Chem.GetAdjacencyMatrix(mol)

                """Transform the above each data of numpy
                to pytorch tensor on a device (i.e., CPU or GPU).
                """
                fingerprints1 = torch.LongTensor(fingerprints1).to(device)
                adjacency1 = torch.FloatTensor(adjacency1).to(device)
                fingerprints2 = torch.LongTensor(fingerprints2).to(device)
                adjacency2 = torch.FloatTensor(adjacency2).to(device)

                if task == 'regression':
                    property = torch.FloatTensor([[float(property)]]).to(device)

                dataset.append((fingerprints1, adjacency1, molecular_size1, fingerprints2, adjacency2, molecular_size2, property))
            
            except:
                non_working_smiles_index.append(data_index)
            
        return dataset

    dataset_train = create_dataset(data_for_gnn[:22000])
    print('Skipped '+ str(len(non_working_smiles_index)) + ' smiles in training data')
    dataset_train, dataset_dev = split_dataset(dataset_train, 0.9)
    dataset_test = create_dataset(data_for_gnn[22000:])

    N_fingerprints = len(fingerprint_dict)

    return dataset_train, dataset_dev, dataset_test, N_fingerprints


In [ ]:
import sys
import timeit

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import roc_auc_score

# import preprocess as pp


class MolecularGraphNeuralNetwork(nn.Module):
    def __init__(self, N_fingerprints, dim, layer_hidden, layer_output):
        super(MolecularGraphNeuralNetwork, self).__init__()
        self.embed_fingerprint = nn.Embedding(N_fingerprints, dim)
        self.W_fingerprint = nn.ModuleList([nn.Linear(dim, dim)
                                            for _ in range(layer_hidden)])
        self.W_output = nn.ModuleList([nn.Linear(dim+dim, dim+dim)
                                       for _ in range(layer_output)])
        if task == 'classification':
            self.W_property = nn.Linear(dim, 2)
        if task == 'regression':
            self.W_property = nn.Linear(dim+dim, 1)

    def pad(self, matrices, pad_value):
        """Pad the list of matrices
        with a pad_value (e.g., 0) for batch processing.
        For example, given a list of matrices [A, B, C],
        we obtain a new matrix [A00, 0B0, 00C],
        where 0 is the zero (i.e., pad value) matrix.
        """
        shapes = [m.shape for m in matrices]
        M, N = sum([s[0] for s in shapes]), sum([s[1] for s in shapes])
        zeros = torch.FloatTensor(np.zeros((M, N))).to(device)
        pad_matrices = pad_value + zeros
        i, j = 0, 0
        for k, matrix in enumerate(matrices):
            m, n = shapes[k]
            pad_matrices[i:i+m, j:j+n] = matrix
            i += m
            j += n
        return pad_matrices

    def update(self, matrix, vectors, layer):
        hidden_vectors = torch.relu(self.W_fingerprint[layer](vectors))
        return hidden_vectors + torch.matmul(matrix, hidden_vectors)

    def sum(self, vectors, axis):
        sum_vectors = [torch.sum(v, 0) for v in torch.split(vectors, axis)]
        return torch.stack(sum_vectors)

    def mean(self, vectors, axis):
        mean_vectors = [torch.mean(v, 0) for v in torch.split(vectors, axis)]
        return torch.stack(mean_vectors)

    def gnn(self, inputs):

        """Cat or pad each input data for batch processing."""
        fingerprints, adjacencies, molecular_sizes = inputs
        fingerprints = torch.cat(fingerprints)
        adjacencies = self.pad(adjacencies, 0)

        """GNN layer (update the fingerprint vectors)."""
        fingerprint_vectors = self.embed_fingerprint(fingerprints)
        for l in range(layer_hidden):
            hs = self.update(adjacencies, fingerprint_vectors, l)
            fingerprint_vectors = F.normalize(hs, 2, 1)  # normalize.

        """Molecular vector by sum or mean of the fingerprint vectors."""
        molecular_vectors = self.sum(fingerprint_vectors, molecular_sizes)
        # molecular_vectors = self.mean(fingerprint_vectors, molecular_sizes)

        return molecular_vectors

    def mlp(self, vectors):
        """Classifier or regressor based on multilayer perceptron."""
        for l in range(layer_output):
#             print(self.W_output[l](vectors))
#             print(self.W_output[l](vectors).shape)
            vectors = torch.relu(self.W_output[l](vectors))
        outputs = self.W_property(vectors)
        return outputs

    def forward_classifier(self, data_batch, train):

        inputs = data_batch[:-1]
        correct_labels = torch.cat(data_batch[-1])

        if train:
            molecular_vectors = self.gnn(inputs)
            predicted_scores = self.mlp(molecular_vectors)
            loss = F.cross_entropy(predicted_scores, correct_labels)
            return loss
        else:
            with torch.no_grad():
                molecular_vectors = self.gnn(inputs)
                predicted_scores = self.mlp(molecular_vectors)
            predicted_scores = predicted_scores.to('cpu').data.numpy()
            predicted_scores = [s[1] for s in predicted_scores]
            correct_labels = correct_labels.to('cpu').data.numpy()
            return predicted_scores, correct_labels

    def forward_regressor(self, data_batch, train):

        inputs_d1 = data_batch[:3]
        inputs_d2 = data_batch[3:-1]
        correct_values = torch.cat(data_batch[-1])

        if train:
            molecular_vectors_d1 = self.gnn(inputs_d1)
            molecular_vectors_d2 = self.gnn(inputs_d2)
#             molecular_vectors = np.concatenate([molecular_vectors_d1,molecular_vectors_d2])
            molecular_vectors = torch.cat((molecular_vectors_d1, molecular_vectors_d2), 1)
            predicted_values = self.mlp(molecular_vectors)
#             print(predicted_values.shape, correct_values.shape, molecular_vectors.shape, molecular_vectors_d1.shape)
            loss = F.mse_loss(predicted_values, correct_values)
            return loss
        else:
            with torch.no_grad():
                molecular_vectors_d1 = self.gnn(inputs_d1)
                molecular_vectors_d2 = self.gnn(inputs_d2)
                molecular_vectors = torch.cat((molecular_vectors_d1, molecular_vectors_d2), 1)
                predicted_values = self.mlp(molecular_vectors)
            predicted_values = predicted_values.to('cpu').data.numpy()
            correct_values = correct_values.to('cpu').data.numpy()
            predicted_values = np.concatenate(predicted_values)
            correct_values = np.concatenate(correct_values)
            return predicted_values, correct_values


class Trainer(object):
    def __init__(self, model):
        self.model = model
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)

    def train(self, dataset):
        np.random.shuffle(dataset)
        N = len(dataset)
        loss_total = 0
        for i in range(0, N, batch_train):
            data_batch = list(zip(*dataset[i:i+batch_train]))
            if task == 'classification':
                loss = self.model.forward_classifier(data_batch, train=True)
            if task == 'regression':
                loss = self.model.forward_regressor(data_batch, train=True)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            loss_total += loss.item()
        return loss_total


class Tester(object):
    def __init__(self, model):
        self.model = model

    def test_regressor(self, dataset):
        N = len(dataset)
        SAE = 0  # sum absolute error.
        for i in range(0, N, batch_test):
            data_batch = list(zip(*dataset[i:i+batch_test]))
            predicted_values, correct_values = self.model.forward_regressor(
                                               data_batch, train=False)
            SAE += sum(np.abs(predicted_values-correct_values))
        MAE = SAE / N  # mean absolute error.
        return MAE

    def save_result(self, result, filename):
        with open(filename, 'a') as f:
            f.write(result + '\n')


In [ ]:
# Run the main code function:


if __name__ == "__main__":
    
    task = 'regression'
    dataset = ''
    radius = 3
    dim = 50
    layer_hidden=6
    layer_output=6

    batch_train=32
    batch_test=32
    lr=1e-4
    lr_decay=0.99
    decay_interval=10
    iteration=5

#     (task, dataset, radius, dim, layer_hidden, layer_output,
#      batch_train, batch_test, lr, lr_decay, decay_interval, iteration,
#      setting) = sys.argv[1:]
    (radius, dim, layer_hidden, layer_output,
     batch_train, batch_test, decay_interval,
     iteration) = map(int, [radius, dim, layer_hidden, layer_output,
                            batch_train, batch_test,
                            decay_interval, iteration])
    lr, lr_decay = map(float, [lr, lr_decay])
#     device = torch.device('cuda')
    # device = torch.device('cpu')
    
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print('The code uses a GPU!')
    else:
        device = torch.device('cpu')
        print('The code uses a CPU...')
#     print('-'*100)

    print('Preprocessing the', dataset, 'dataset.')
    print('Just a moment......')
    (dataset_train, dataset_dev, dataset_test,
     N_fingerprints) = create_datasets(task, data_for_gnn, radius, device)
    print('-'*100)

    print('The preprocess has finished!')
    print('# of training data samples:', len(dataset_train))
    print('# of development data samples:', len(dataset_dev))
    print('# of test data samples:', len(dataset_test))
    print('-'*100)

    

The code uses a GPU!
Preprocessing the  dataset.
Just a moment......
<class 'pandas.core.frame.DataFrame'>
(23052, 3)
Skipped 1639 smiles in training data
----------------------------------------------------------------------------------------------------
The preprocess has finished!
# of training data samples: 18324
# of development data samples: 2037
# of test data samples: 975
----------------------------------------------------------------------------------------------------


In [ ]:
print(N_fingerprints)

NameError: ignored

In [ ]:
dim = 50
iteration=100
print('Creating a model.')
torch.manual_seed(1234)
model = MolecularGraphNeuralNetwork(
        N_fingerprints, dim, layer_hidden, layer_output).to(device)
trainer = Trainer(model)
tester = Tester(model)
print('# of model parameters:',
      sum([np.prod(p.size()) for p in model.parameters()]))
print('-'*100)

#     file_result = '../output/result--' + setting + '.txt'
file_result = train_data_path + '/setting1_run.txt'

if task == 'classification':
    result = 'Epoch\tTime(sec)\tLoss_train\tAUC_dev\tAUC_test'
if task == 'regression':
    result = 'Epoch\tTime(sec)\tLoss_train\tMAE_dev\tMAE_test'

with open(file_result, 'w') as f:
    f.write(result + '\n')

print('Start training.')
  print('The result is saved in the output directory every epoch!')

all_loss_train = []


Creating a model.
# of model parameters: 180251
----------------------------------------------------------------------------------------------------
Start training.
The result is saved in the output directory every epoch!


In [ ]:
# np.random.seed(1234)

start = timeit.default_timer()

for epoch in range(iteration):

    # Dont decay lr
    # if epoch % decay_interval == 0:
    #     trainer.optimizer.param_groups[0]['lr'] *= lr_decay

    loss_train = trainer.train(dataset_train)

    if task == 'regression':
        prediction_dev = tester.test_regressor(dataset_dev)
        prediction_test = tester.test_regressor(dataset_test)

    time = timeit.default_timer() - start

    if epoch == 1:
        minutes = time * iteration / 60
        hours = int(minutes / 60)
        minutes = int(minutes - 60 * hours)
        print('The training will finish in about',
              hours, 'hours', minutes, 'minutes.')
        print('-'*100)
        print(result)

    result = '\t'.join(map(str, [epoch, time, loss_train,
                                 prediction_dev, prediction_test]))
    tester.save_result(result, file_result)

    print(result)
    all_loss_train.append(loss_train)
    epoch += 1

0	29.9315525539987	551.2848766446114	0.5832435384504295	0.622982444260747
The training will finish in about 1 hours 39 minutes.
----------------------------------------------------------------------------------------------------
0	29.9315525539987	551.2848766446114	0.5832435384504295	0.622982444260747
1	59.74462474199936	482.29436922073364	0.5516031286704355	0.5940023453896627
2	93.15232365499833	470.91934306174517	0.5515762088119903	0.5941910936845801
3	123.22701256599976	468.1806314140558	0.5469057483271232	0.5877234701239146
4	152.79523120799968	463.9287303239107	0.5521752930273268	0.5953469445155217
5	183.2421351239991	461.8805976808071	0.5517123117581171	0.5960749223828316
6	212.86336595600005	462.88788129389286	0.5507626289416746	0.5937085735148344
7	243.66531294299966	460.2061123922467	0.5461046916236538	0.5854636483257397
8	273.0783623309999	457.9523325264454	0.5471832461508378	0.5889735054702331
9	302.35882433199913	457.5110407471657	0.546294561098524	0.589263603402636
10	331.

In [ ]:
len(all_loss_train)

100

In [ ]:
# torch.save(model.state_dict(), train_data_path +'/model_100iter_r3_scaled_colab_hr1')
#Later to restore:
aaaa = model.load_state_dict(torch.load(train_data_path +'/model_100iter_r3_scaled_colab_hr1.h5'))
# model.eval()


In [ ]:
# !pip install hiddenlayer
!pip install torchinfo

In [ ]:
# np.random.seed(1234)

# start = timeit.default_timer()

# epoch_batch_size = int( len(dataset_train)/iteration )
# batch_start_nd = 0
# batch_end_nd = epoch_batch_size

# iteration = 3

# for epoch in range(iteration):
#     print(batch_start_nd, batch_end_nd)
#     epoch += 1
#     if epoch % decay_interval == 0:
#         trainer.optimizer.param_groups[0]['lr'] *= lr_decay

#     if(epoch != iteration - 1):
#       loss_train = trainer.train(dataset_train[batch_start_nd : batch_end_nd])

#     if(epoch == iteration - 1):
#       loss_train = trainer.train(dataset_train[batch_start_nd :])

#     if task == 'regression':
#         prediction_dev = tester.test_regressor(dataset_dev)
#         prediction_test = tester.test_regressor(dataset_test)

#     time = timeit.default_timer() - start

#     if epoch == 1:
#         minutes = time * iteration / 60
#         hours = int(minutes / 60)
#         minutes = int(minutes - 60 * hours)
#         print('The training will finish in about',
#               hours, 'hours', minutes, 'minutes.')
#         print('-'*100)
#         print(result)

#     result = '\t'.join(map(str, [epoch, time, loss_train,
#                                  prediction_dev, prediction_test]))
#     tester.save_result(result, file_result)

#     print(result)
#     all_loss_train.append(loss_train)

#     batch_start_nd = batch_end_nd
#     batch_end_nd = batch_start_nd + epoch_batch_size


0 9166
The training will finish in about 0 hours 1 minutes.
----------------------------------------------------------------------------------------------------
0	33.53304521900009	242657.7946395874	12.522179454550418	12.134580989981758
1	25.38518947099965	117509.69805145264	12.588177393041057	12.431383783777843
9166 18332
2	43.331842485999914	124835.9345703125	12.743936663626103	12.406385482467071
18332 27498
3	46.3504230210001	0	12.743936663626103	12.406385482467071


#### updated version:

In [2]:
train_data_path = "/content/drive/My Drive/Projects and research stuffs/Drug Synergy/Dataset for GNN"
main_data = pd.read_csv(train_data_path+'/final_data.csv')
print(main_data.shape)
main_data.head()

from sklearn.preprocessing import StandardScaler
data_for_gnn = main_data[['drug_a_structure', 'drug_b_structure', 'cell_line', 'synergy']].copy()
data_for_gnn = data_for_gnn.sample(frac=1).reset_index().drop(['index'], 1)

scaler = StandardScaler().fit(data_for_gnn['synergy'].values.reshape(-1,1))
scaled_ops = scaler.transform(data_for_gnn['synergy'].values.reshape(-1,1))
data_for_gnn['synergy'] = scaled_ops
data_for_gnn.head()


In [9]:
# ### Importing dataset using their script

# import requests
# import os
# import zlib


# def _download_file(response, filename):
#     with open(filename, 'wb') as f:
#         for chunk in response.iter_content(chunk_size=1024):
#             f.write(chunk)


# def _download_and_decompress_file(response, filename):
#     decompressor = zlib.decompressobj(16 + zlib.MAX_WBITS)
#     filename = filename[:-3]
#     with open(filename, 'w+') as f:
#         while True:
#             chunk = response.raw.read(1024)
#             if not chunk:
#                 break
#             string = decompressor.decompress(chunk)
#             f.write(string)


# def download_datasets(selected_datasets, selected_downloads, decompress=False):
#     for dataset, path in selected_datasets:
#         if not os.path.exists(dataset):
#             os.mkdir(dataset)

#         for downloadable in selected_downloads:
#             url = 'https://maayanlab.cloud/static/hdfs/harmonizome/data/%s/%s' %\
#                   (path, downloadable)
#             response = requests.get(url, stream=True)
#             filename = '%s/%s' % (dataset, downloadable)

#             # Not every dataset has all downloadables.
#             if response.status_code != 200:
#                 continue

#             if decompress and 'txt.gz' in filename:
#                 _download_and_decompress_file(response, filename)
#             else:
#                 _download_file(response, filename)

#         print('%s downloaded.' % dataset)


# if __name__ == '__main__':
#     # Uncomment a dataset to download it.
#     download_datasets([
# #         ('Achilles Cell Line Gene Essentiality Profiles', 'achilles'),
# #         ('Allen Brain Atlas Adult Human Brain Tissue Gene Expression Profiles', 'brainatlasadulthuman'),
# #         ('Allen Brain Atlas Adult Mouse Brain Tissue Gene Expression Profiles', 'brainatlasadultmouse'),
# #         ('Allen Brain Atlas Developing Human Brain Tissue Gene Expression Profiles by Microarray', 'brainatlasdevelopmentalhumanmicroarray'),
# #         ('Allen Brain Atlas Developing Human Brain Tissue Gene Expression Profiles by RNA-seq', 'brainatlasdevelopmentalhumanrnaseq'),
# #         ('Allen Brain Atlas Prenatal Human Brain Tissue Gene Expression Profiles', 'brainatlasprenatalhuman'),
# #         ('BIND Biomolecular Interactions', 'bind'),
# #         ('BioGPS Cell Line Gene Expression Profiles', 'biogpsnci60'),
# #         ('BioGPS Human Cell Type and Tissue Gene Expression Profiles', 'biogpshuman'),
# #         ('BioGPS Mouse Cell Type and Tissue Gene Expression Profiles', 'biogpsmouse'),
# #         ('BioGRID Protein-Protein Interactions', 'biogrid'),
# #         ('Biocarta Pathways', 'biocarta'),
#         ('CCLE Cell Line Gene CNV Profiles', 'cclecnv'),
#         ('CCLE Cell Line Gene Expression Profiles', 'cclemrna'),
#         ('CCLE Cell Line Gene Mutation Profiles', 'cclemut'),
# #         ('CHEA Transcription Factor Binding Site Profiles', 'chea'),
# #         ('CHEA Transcription Factor Targets', 'cheappi'),
# #         ('CMAP Signatures of Differentially Expressed Genes for Small Molecules', 'cmap'),
# #         ('COMPARTMENTS Curated Protein Localization Evidence Scores', 'jensencompartmentcurated'),
# #         ('COMPARTMENTS Experimental Protein Localization Evidence Scores', 'jensencompartmentexpts'),
# #         ('COMPARTMENTS Text-mining Protein Localization Evidence Scores', 'jensencompartmenttextmining'),
# #         ('CORUM Protein Complexes', 'corum'),
# #         ('COSMIC Cell Line Gene CNV Profiles', 'cosmiccnv'),
# #         ('COSMIC Cell Line Gene Mutation Profiles', 'cosmicmut'),
# #         ('CTD Gene-Chemical Interactions', 'ctdchemical'),
# #         ('CTD Gene-Disease Associations', 'ctddisease'),
# #         ('ClinVar SNP-Phenotype Associations', 'clinvar'),
# #         ('Combined Pathways Pathways', 'combinedpathways'),
# #         ('dbGAP Gene-Trait Associations', 'dbgap'),
# #         ('DEPOD Substrates of Phosphatases', 'depod'),
# #         ('DIP Protein-Protein Interactions', 'dip'),
# #         ('DISEASES Curated Gene-Disease Assocation Evidence Scores', 'jensendiseasecurated'),
# #         ('DISEASES Experimental Gene-Disease Assocation Evidence Scores', 'jensendiseaseexpts'),
# #         ('DISEASES Text-mining Gene-Disease Assocation Evidence Scores', 'jensendiseasetextmining'),
# #         ('DrugBank Drug Targets', 'drugbank'),
# #         ('ENCODE Histone Modification Site Profiles', 'encodehm'),
# #         ('ENCODE Transcription Factor Binding Site Profiles', 'encodetf'),
# #         ('ENCODE Transcription Factor Targets', 'encodetfppi'),
# #         ('ESCAPE Omics Signatures of Genes and Proteins for Stem Cells', 'escape'),
# #         ('GAD Gene-Disease Associations', 'gad'),
# #         ('GAD High Level Gene-Disease Associations', 'gadhighlevel'),
# #         ('GDSC Cell Line Gene Expression Profiles', 'gdsc'),
# #         ('GEO Signatures of Differentially Expressed Genes for Diseases', 'geodisease'),
# #         ('GEO Signatures of Differentially Expressed Genes for Gene Perturbations', 'geogene'),
# #         ('GEO Signatures of Differentially Expressed Genes for Kinase Perturbations', 'geokinase'),
# #         ('GEO Signatures of Differentially Expressed Genes for Small Molecules', 'geochemical'),
# #         ('GEO Signatures of Differentially Expressed Genes for Transcription Factor Perturbations', 'geotf'),
# #         ('GEO Signatures of Differentially Expressed Genes for Viral Infections', 'geovirus'),
# #         ('GO Biological Process Annotations', 'gobp'),
# #         ('GO Cellular Component Annotations', 'gocc'),
# #         ('GO Molecular Function Annotations', 'gomf'),
# #         ('GTEx Tissue Gene Expression Profiles', 'gtextissue'),
# #         ('GTEx Tissue Sample Gene Expression Profiles', 'gtexsample'),
# #         ('GTEx eQTL', 'gtexeqtl'),
# #         ('GWAS Catalog SNP-Phenotype Associations', 'gwascatalog'),
# #         ('GWASdb SNP-Disease Associations', 'gwasdbdisease'),
# #         ('GWASdb SNP-Phenotype Associations', 'gwasdbphenotype'),
# #         ('GeneRIF Biological Term Annotations', 'generif'),
# #         ('GeneSigDB Published Gene Signatures', 'genesigdb'),
# #         ('Graph of Medicine EHR Text-mining Clinical Term Annotations', 'graphofmedicine'),
# #         ('Guide to Pharmacology Chemical Ligands of Receptors', 'guidetopharmchemical'),
# #         ('Guide to Pharmacology Protein Ligands of Receptors', 'guidetopharmprotein'),
# #         ('HMDB Metabolites of Enzymes', 'hmdb'),
# #         ('HPA Cell Line Gene Expression Profiles', 'hpacelllines'),
# #         ('HPA Tissue Gene Expression Profiles', 'hpatissuesmrna'),
# #         ('HPA Tissue Protein Expression Profiles', 'hpatissuesprotein'),
# #         ('HPA Tissue Sample Gene Expression Profiles', 'hpasamples'),
# #         ('HPM Cell Type and Tissue Protein Expression Profiles', 'hpm'),
# #         ('HPO Gene-Disease Associations', 'hpo'),
# #         ('HPRD Protein-Protein Interactions', 'hprd'),
# #         ('Heiser et al., PNAS, 2011 Cell Line Gene Expression Profiles', 'heiser'),
# #         ('HuGE Navigator Gene-Phenotype Associations', 'hugenavigator'),
# #         ('Hub Proteins Protein-Protein Interactions', 'hubs'),
# #         ('HumanCyc Biomolecular Interactions', 'humancycppi'),
# #         ('HumanCyc Pathways', 'humancyc'),
# #         ('IntAct Biomolecular Interactions', 'intact'),
# #         ('InterPro Predicted Protein Domain Annotations', 'interpro'),
# #         ('JASPAR Predicted Transcription Factor Targets', 'jasparpwm'),
# #         ('KEA Substrates of Kinases', 'kea'),
# #         ('KEGG Biomolecular Interactions', 'keggppi'),
# #         ('KEGG Pathways', 'kegg'),
# #         ('Kinativ Kinase Inhibitor Bioactivity Profiles', 'kinativ'),
# #         ('KinomeScan Kinase Inhibitor Targets', 'kinomescan'),
# #         ('Klijn et al., Nat. Biotechnol., 2015 Cell Line Gene CNV Profiles', 'klijncnv'),
# #         ('Klijn et al., Nat. Biotechnol., 2015 Cell Line Gene Expression Profiles', 'klijnmrna'),
# #         ('Klijn et al., Nat. Biotechnol., 2015 Cell Line Gene Mutation Profiles', 'klijnmut'),
# #         ('LINCS L1000 CMAP Signatures of Differentially Expressed Genes for Gene Knockdowns', 'lincscmapgene'),
# #         ('LINCS L1000 CMAP Signatures of Differentially Expressed Genes for Small Molecules', 'lincscmapchemical'),
# #         ('LOCATE Curated Protein Localization Annotations', 'locate'),
# #         ('LOCATE Predicted Protein Localization Annotations', 'locatepredicted'),
# #         ('MPO Gene-Phenotype Associations', 'mgimpo'),
# #         ('MSigDB Cancer Gene Co-expression Modules', 'msigdbcomp'),
# #         ('MSigDB Signatures of Differentially Expressed Genes for Cancer Gene Perturbations', 'msigdbonc'),
# #         ('MiRTarBase microRNA Targets', 'mirtarbase'),
# #         ('MotifMap Predicted Transcription Factor Targets', 'motifmap'),
# #         ('NURSA Protein Complexes', 'nursa'),
# #         ('NURSA Protein-Protein Interactions', 'nursappi'),
# #         ('OMIM Gene-Disease Associations', 'omim'),
# #         ('PANTHER Biomolecular Interactions', 'pantherppi'),
# #         ('PANTHER Pathways', 'panther'),
# #         ('PID Biomolecular Interactions', 'pidppi'),
# #         ('PID Pathways', 'pid'),
# #         ('Pathway Commons Protein-Protein Interactions', 'pc'),
# #         ('PhosphoSitePlus Phosphosite-Disease Associations', 'phosphositeplusdisease'),
# #         ('PhosphoSitePlus Substrates of Kinases', 'phosphositeplus'),
# #         ('Phosphosite Textmining Biological Term Annotations', 'phosphositetextmining'),
# #         ('ProteomicsDB Cell Type and Tissue Protein Expression Profiles', 'proteomicsdb'),
# #         ('Reactome Biomolecular Interactions', 'reactomeppi'),
# #         ('Reactome Pathways', 'reactome'),
# #         ('Recon X Predicted Biomolecular Interactions', 'reconx'),
# #         ('Roadmap Epigenomics Cell and Tissue DNA Accessibility Profiles', 'epigenomicsdnaaccessibility'),
# #         ('Roadmap Epigenomics Cell and Tissue DNA Methylation Profiles', 'epigenomicsdnamethylation'),
# #         ('Roadmap Epigenomics Cell and Tissue Gene Expression Profiles', 'epigenomicsmrna'),
# #         ('Roadmap Epigenomics Histone Modification Site Profiles', 'epigenomicshm'),
# #         ('SILAC Phosphoproteomics Signatures of Differentially Phosphorylated Proteins for Drugs', 'silacdrug'),
# #         ('SILAC Phosphoproteomics Signatures of Differentially Phosphorylated Proteins for Gene Perturbations', 'silacgene'),
# #         ('SILAC Phosphoproteomics Signatures of Differentially Phosphorylated Proteins for Protein Ligands', 'silacligand'),
# #         ('SNPedia SNP-Phenotype Associations', 'snpedia'),
# #         ('TCGA Signatures of Differentially Expressed Genes for Tumors', 'tcga'),
# #         ('TISSUES Curated Tissue Protein Expression Evidence Scores', 'jensentissuecurated'),
# #         ('TISSUES Experimental Tissue Protein Expression Evidence Scores', 'jensentissueexpts'),
# #         ('TISSUES Text-mining Tissue Protein Expression Evidence Scores', 'jensentissuetextmining'),
# #         ('TRANSFAC Curated Transcription Factor Targets', 'transfac'),
# #         ('TRANSFAC Predicted Transcription Factor Targets', 'transfacpwm'),
# #         ('TargetScan Predicted Conserved microRNA Targets', 'targetscan'),
# #         ('TargetScan Predicted Nonconserved microRNA Targets', 'targetscannonconserved'),
# #         ('Virus MINT Protein-Viral Protein Interactions', 'virusmintppi'),
# #         ('Virus MINT Protein-Virus Interactions', 'virusmint'),
# #         ('Wikipathways Pathways', 'wikipathways'),
#     ], [
#         'gene_attribute_matrix.txt.gz',
#         'gene_attribute_edges.txt.gz',
# #         'gene_set_library_crisp.txt.gz',
# #         'gene_set_library_up_crisp.txt.gz',
# #         'gene_set_library_dn_crisp.txt.gz',
# #         'attribute_set_library_crisp.txt.gz',
# #         'attribute_set_library_up_crisp.txt.gz',
# #         'attribute_set_library_dn_crisp.txt.gz',
# #         'gene_similarity_matrix_cosine.txt.gz',
# #         'attribute_similarity_matrix_cosine.txt.gz',
# #         'gene_list_terms.txt.gz',
# #         'attribute_list_entries.txt.gz',
#         'processing_script.m'
#     ])

In [10]:
# main_data.head()

#### Pre-processing gene expression attribute matrix and edge matrix

In [9]:
ccle_attr = pd.read_csv('/content/drive/My Drive/Projects and research stuffs/Drug Synergy/Dataset for GNN/CCLE Cell Line Gene Expression Profiles/gene_attribute_matrix.txt.gz', sep='\t')
print(ccle_attr.shape)

(18027, 1040)


In [17]:
ccle_attr.head()

,#,#.1,CellLine,CHL1,HMCB,HS852T,HS695T,A101D,HS294T,SNU466,...,HEL9217,HEL,UT7,SET2,MOLM16,KU812,TF1,MEG01,KYO1,K562
0,#,#,Tissue,skin,skin,skin,skin,skin,skin,central nervous system,...,haematopoietic and lymphoid tissue,haematopoietic and lymphoid tissue,haematopoietic and lymphoid tissue,haematopoietic and lymphoid tissue,haematopoietic and lymphoid tissue,haematopoietic and lymphoid tissue,haematopoietic and lymphoid tissue,haematopoietic and lymphoid tissue,haematopoietic and lymphoid tissue,haematopoietic and lymphoid tissue
1,GeneSym,NaN,GeneID/NA,na,na,na,na,na,na,na,...,na,na,na,na,na,na,na,na,na,na
2,FBN1,na,2200,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,...,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000
3,ITGBL1,na,9358,-0.000000,-0.000000,0.000000,-0.000000,-0.000000,-0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000
4,LRP1,na,4035,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000


In [43]:
# Read attribute matrix
ccle_attr = pd.read_csv('/content/drive/My Drive/Projects and research stuffs/Drug Synergy/Dataset for GNN/CCLE Cell Line Gene Expression Profiles/gene_attribute_matrix.txt.gz', sep='\t')
print(ccle_attr.shape)
cell2ind = pd.DataFrame(columns = ['index','Cell_lines'])
cell2ind['Cell_lines'] = list(ccle_attr.columns[3:])
cell2ind['index'] =  list(cell2ind.index)
print(cell2ind.head())
gene2ind = pd.DataFrame(columns = ['index','Genes', 'Gene_id'])
gene2ind['Genes'] =  list(ccle_attr['#'].values[2:])
gene2ind['index'] =  list(gene2ind.index)
gene2ind['Gene_id'] =  list(ccle_attr['CellLine'].values[2:])

print(gene2ind.head())
cell2exp = pd.DataFrame(columns = ['expression'])
l = []
for cells_ind in range(len(cell2ind['Cell_lines'])):
    l.append(list(ccle_attr[cell2ind['Cell_lines'].iloc[cells_ind]].values[2:]))
    
cell2exp['expression'] = l
print(cell2exp.head())

(18027, 1040)
   index Cell_lines
0      0       CHL1
1      1       HMCB
2      2     HS852T
3      3     HS695T
4      4      A101D
   index   Genes Gene_id
0      0    FBN1    2200
1      1  ITGBL1    9358
2      2    LRP1    4035
3      3   LTBP2    4053
4      4   PARVA   55742
                                          expression
0  [0.000000, -0.000000, -0.000000, -0.000000, -0...
1  [0.000000, -0.000000, -0.000000, -0.000000, -0...
2  [0.000000, 0.000000, 0.000000, -0.000000, 0.00...
3  [-0.000000, -0.000000, 0.000000, 0.000000, 0.0...
4  [0.000000, -0.000000, 0.000000, -0.000000, 0.0...


In [44]:
print(gene2ind.shape)

(18025, 3)


In [45]:
pp_int = pd.read_csv('/content/drive/My Drive/Projects and research stuffs/Drug Synergy/Dataset for GNN/PP-Pathways_ppi.csv.gz', header=None)
pp_int.columns = ['Protein1', 'Protein2']
pp_int.head()

,Protein1,Protein2
0,1394,2778
1,6331,17999
2,122704,54460
3,2597,2911
4,4790,79155


In [46]:
# This dataset has proper gene-gene interactions
# pp_int[pp_int['Protein1'] == 2200][:2]

In [47]:
# If any genes are cell line ids, remove them
cell_ids = cell2ind['Cell_lines'].unique()
to_drop = []
for i in range(len(gene2ind)):
  if(gene2ind.iloc[i]['Genes'] in cell_ids):
    to_drop.append(i)

print(len(to_drop))

gene2ind = gene2ind.drop(to_drop, axis = 0)

6


In [57]:
gene2ind.shape

(15000, 3)

In [49]:
# Check and take only intersection of gene ids

# Create set of all values from both p1 and p2 columns
p1p2 = set()
# for i in range(len(pp_int)):
#   p1p2.add(pp_int['Protein1'].iloc[i])
#   p1p2.add(pp_int['Protein2'].iloc[i])

p1p2 = set(np.concatenate((pp_int['Protein1'].unique(), pp_int['Protein2'].unique())))
print('Total gene ids in pp_int dataset: ', len(p1p2))

# Check if any gene ids from deepsynergy dataset is missing in this p1p2
common_gene_ids = []
uncommon_gene_ids = []

for i in range(len(gene2ind)):
  if(gene2ind.iloc[i]['Gene_id'] in p1p2):
    common_gene_ids.append(gene2ind.iloc[i]['Gene_id'])
  else:
    uncommon_gene_ids.append(gene2ind.iloc[i]['Gene_id'])

print('Total Common intersecting gene ids:', len(common_gene_ids), ' and Total uncommon non-intersecting gene ids: ', len(uncommon_gene_ids))



Total gene ids in pp_int dataset:  21557
Total Common intersecting gene ids: 14994  and Total uncommon non-intersecting gene ids:  3025


In [50]:
# Take only 5k genes (avoid computational problems )
# final_gene_ids = common_gene_ids[:10000]



In [51]:
ccle_attr.head()

,#,#.1,CellLine,CHL1,HMCB,HS852T,HS695T,A101D,HS294T,SNU466,...,HEL9217,HEL,UT7,SET2,MOLM16,KU812,TF1,MEG01,KYO1,K562
0,#,#,Tissue,skin,skin,skin,skin,skin,skin,central nervous system,...,haematopoietic and lymphoid tissue,haematopoietic and lymphoid tissue,haematopoietic and lymphoid tissue,haematopoietic and lymphoid tissue,haematopoietic and lymphoid tissue,haematopoietic and lymphoid tissue,haematopoietic and lymphoid tissue,haematopoietic and lymphoid tissue,haematopoietic and lymphoid tissue,haematopoietic and lymphoid tissue
1,GeneSym,NaN,GeneID/NA,na,na,na,na,na,na,na,...,na,na,na,na,na,na,na,na,na,na
2,FBN1,na,2200,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,...,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000
3,ITGBL1,na,9358,-0.000000,-0.000000,0.000000,-0.000000,-0.000000,-0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000
4,LRP1,na,4035,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000


#### Re-read cell_attr, this time ignore uncommon gene ids

In [52]:
# Remove rows with uncommon gene ids
to_drop = []
for i in range(2, len(ccle_attr)):
  if(ccle_attr.iloc[i]['CellLine'] in uncommon_gene_ids):
    to_drop.append(i)

print('dropping ', len(to_drop), ' genes')
ccle_attr = ccle_attr.drop(to_drop, axis = 0)
print(ccle_attr.shape)


dropping  3025  genes
(15002, 1040)


In [53]:
# Drop from pp_int dataset
to_drop = set()
for i in range(len(pp_int)):
  if(pp_int.iloc[i]['Protein1'] in uncommon_gene_ids):
    to_drop.add(i)
  if(pp_int.iloc[i]['Protein2'] in uncommon_gene_ids):
    to_drop.add(i)

to_drop = list(to_drop)
pp_int = pp_int.drop(to_drop, axis = 0)
print(pp_int.shape)



(342353, 2)


In [54]:
# Save these two datasets, no need to run again
# ccle_attr.to_csv('/content/drive/My Drive/Projects and research stuffs/Drug Synergy/Dataset for GNN/ccle_attr_updated.csv', index=False)
# pp_int.to_csv('/content/drive/My Drive/Projects and research stuffs/Drug Synergy/Dataset for GNN/pp_int_updated.csv', index=False)

# ccle_attr = pd.read_csv('/content/drive/My Drive/Projects and research stuffs/Drug Synergy/Dataset for GNN/ccle_attr_updated.csv')
# pp_int = pd.read_csv('/content/drive/My Drive/Projects and research stuffs/Drug Synergy/Dataset for GNN/pp_int_updated.csv')

In [55]:

cell2ind = pd.DataFrame(columns = ['index','Cell_lines'])
cell2ind['Cell_lines'] = list(ccle_attr.columns[3:])
cell2ind['index'] =  list(cell2ind.index)
print(cell2ind.head())
gene2ind = pd.DataFrame(columns = ['index','Genes', 'Gene_id'])
gene2ind['Genes'] =  list(ccle_attr['#'].values[2:])
gene2ind['index'] =  list(gene2ind.index)
gene2ind['Gene_id'] =  list(ccle_attr['CellLine'].values[2:])

print(gene2ind.head())
cell2exp = pd.DataFrame(columns = ['expression'])
l = []
for cells_ind in range(len(cell2ind['Cell_lines'])):
    l.append(list(ccle_attr[cell2ind['Cell_lines'].iloc[cells_ind]].values[2:]))
    
cell2exp['expression'] = l
print(cell2exp.head())

   index Cell_lines
0      0       CHL1
1      1       HMCB
2      2     HS852T
3      3     HS695T
4      4      A101D
   index    Genes  Gene_id
0      0      LBH    81606
1      1     GLI2     2736
2      2    PAPPA     5069
3      3  SLC30A4     7782
4      4    JAZF1   221895
                                          expression
0  [-0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -...
1  [-0.0, 1.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0...
2  [0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 0.0...
3  [-0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -...
4  [-0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0...


In [61]:
gene2ind.shape

(15000, 3)

In [62]:
cell2ind['expression'] = cell2exp['expression'].copy()
cell2ind = cell2ind.rename(columns = {'Cell_lines':'cell_line'})
cell2ind.head()

from sklearn.preprocessing import StandardScaler

data_for_gnn_gcn = main_data[['drug_a_structure', 'drug_b_structure', 'cell_line', 'synergy']][:10000].copy()
scaler = StandardScaler().fit(data_for_gnn_gcn['synergy'].values.reshape(-1,1))
scaled_ops = scaler.transform(data_for_gnn_gcn['synergy'].values.reshape(-1,1))
data_for_gnn_gcn['synergy'] = scaled_ops
data_for_gnn_gcn = data_for_gnn_gcn.sample(frac=1).reset_index().drop(['index'], 1)
data_for_gnn_gcn.head()

,drug_a_structure,drug_b_structure,cell_line,synergy
0,COc1cccc2c1C(=O)c1c(O)c3c(c(O)c1C2=O)CC(O)(C(=...,CCN(CC)CCNC(=O)c1c(C)[nH]c(C=C2C(=O)Nc3ccc(F)c...,UWB1289,-0.353791
1,C=CCn1c(=O)c2cnc(Nc3ccc(N4CCN(C)CC4)cc3)nc2n1-...,Cn1cc(-c2cnn3c(N)c(Br)c(C4CCCNC4)nc23)cn1,UACC62,1.684066
2,O=P1(N(CCCl)CCCl)NCCCO1,O=C(CCCCCCC(=O)Nc1ccccc1)NO,UACC62,0.027459
3,CS(=O)(=O)CCNCc1ccc(-c2ccc3ncnc(Nc4ccc(OCc5ccc...,Cn1c(=O)n(-c2ccc(C(C)(C)C#N)cc2)c2c3cc(-c4cnc5...,NCIH2122,2.094920
4,N.N.O=C(O)C1(C(=O)O)CCC1.[Pt],N#Cc1ccc(Cn2cncc2CN2CCN(c3cccc(Cl)c3)C(=O)C2)cc1,VCAP,-0.066320


In [76]:
l = []

for i in range(len(data_for_gnn_gcn)):
  cell_name = data_for_gnn_gcn.iloc[i]['cell_line']
  # print(cell_name)
  try:
    expr = np.array(cell2ind[cell2ind['cell_line'] == cell_name]['expression'].values[0], dtype='float32')
  except:
    expr = [float(0) for j in range(15000)]
  l.append(expr)
  # new_data.append({ new_data.columns[0]:main_data.iloc[i][new_data.columns[0]], new_data.columns[1]:main_data.iloc[i][new_data.columns[1]], new_data.columns[2]:main_data.iloc[i][new_data.columns[2]], new_data.columns[3]:main_data.iloc[i][new_data.columns[3]] }, ignore_index = True)

data_for_gnn_gcn['cell_line_expr'] = l



In [77]:
data_for_gnn_gcn.head()
import gc
gc.collect()

322

#### Calculate adjacency matrix Using the gene protein-protein interactions

### NOT YET WORKING

In [79]:
pp_int.head()

,Protein1,Protein2
0,1394,2778
1,6331,17999
2,122704,54460
3,2597,2911
4,4790,79155


In [89]:
# Brute-force method ( for some reason sparse_matrixl)
# for i in range(len(pp_int)):
# len( set(np.concatenate((pp_int['Protein1'].unique(), pp_int['Protein2'].unique()))) )

21557

In [118]:
# ## Create dummy adjacency matrix for now
# import scipy.sparse as sp
# adj = sp.coo_matrix((len(data_for_gnn_gcn), len(data_for_gnn_gcn)), dtype=np.float32).toarray()
# # type(sp.csr_matrix(adj))

# # build symmetric adjacency matrix
# # adj = adj + np.matmul(adj.T, adj.T > adj) - np.matmul(adj, adj.T > adj)
# # adj
# #
# def get_adjacency_matrix(df):
#   df = pd.crosstab(df.Protein1, df.Protein2)
#   idx = df.columns.union(df.index)
#   df = df.reindex(index = idx, columns=idx, fill_value=0)

# csc = sp.csc_matrix((np.ones_like(df['A']), (df['A'],df['B'])))
# csc.toarray()

# # Doing this in sparse matrix multiplication
# adj_sp = sp.csr_matrix(adj)
# adj_sp = adj_sp + adj_sp.T.multiply(adj_sp.T > adj_sp) - adj_sp.multiply(adj_sp.T > adj_sp)

In [85]:
G_di = nx.from_pandas_edgelist(pp_int, 'Protein1', 'Protein2', create_using=nx.DiGraph())
A_di = nx.to_scipy_sparse_matrix(G_di)
A_di.todense().shape


(21557, 21557)

In [88]:
len(pp_int['Protein2'].unique())

19341

In [80]:
import networkx as nx
G=nx.from_pandas_edgelist(pp_int, 'Protein1', 'Protein2')
type(G)

networkx.classes.graph.Graph

In [81]:
A = nx.to_scipy_sparse_matrix(G)
A.shape

(21557, 21557)

In [ ]:
# pd.crosstab(pp_int.Protein1, pp_int.Protein2)

In [119]:
import gc
gc.collect()
adj_sp.todense().shape

(10000, 10000)

In [79]:
# pp_int_path = '/content/drive/My Drive/Projects and research stuffs/Drug Synergy/Dataset for GNN/Hsapi20170205.mif25.gz'
# import defusedxml.ElementTree as ET
# ET.parse(pp_int_path)


In [ ]:

def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx



def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [ ]:
#### GNN combined with GCN
from collections import defaultdict

import numpy as np

from rdkit import Chem

import torch


def create_atoms(mol, atom_dict):
    """Transform the atom types in a molecule (e.g., H, C, and O)
    into the indices (e.g., H=0, C=1, and O=2).
    Note that each atom index considers the aromaticity.
    """
    atoms = [a.GetSymbol() for a in mol.GetAtoms()]
    for a in mol.GetAromaticAtoms():
        i = a.GetIdx()
        atoms[i] = (atoms[i], 'aromatic')
    atoms = [atom_dict[a] for a in atoms]
    return np.array(atoms)


def create_ijbonddict(mol, bond_dict):
    """Create a dictionary, in which each key is a node ID
    and each value is the tuples of its neighboring node
    and chemical bond (e.g., single and double) IDs.
    """
    i_jbond_dict = defaultdict(lambda: [])
    for b in mol.GetBonds():
        i, j = b.GetBeginAtomIdx(), b.GetEndAtomIdx()
        bond = bond_dict[str(b.GetBondType())]
        i_jbond_dict[i].append((j, bond))
        i_jbond_dict[j].append((i, bond))
    return i_jbond_dict


def extract_fingerprints(radius, atoms, i_jbond_dict,
                         fingerprint_dict, edge_dict):
    """Extract the fingerprints from a molecular graph
    based on Weisfeiler-Lehman algorithm.
    """

    if (len(atoms) == 1) or (radius == 0):
        nodes = [fingerprint_dict[a] for a in atoms]

    else:
        nodes = atoms
        i_jedge_dict = i_jbond_dict

        for _ in range(radius):

            """Update each node ID considering its neighboring nodes and edges.
            The updated node IDs are the fingerprint IDs.
            """
            nodes_ = []
            for i, j_edge in i_jedge_dict.items():
                neighbors = [(nodes[j], edge) for j, edge in j_edge]
                fingerprint = (nodes[i], tuple(sorted(neighbors)))
                nodes_.append(fingerprint_dict[fingerprint])

            """Also update each edge ID considering
            its two nodes on both sides.
            """
            i_jedge_dict_ = defaultdict(lambda: [])
            for i, j_edge in i_jedge_dict.items():
                for j, edge in j_edge:
                    both_side = tuple(sorted((nodes[i], nodes[j])))
                    edge = edge_dict[(both_side, edge)]
                    i_jedge_dict_[i].append((j, edge))

            nodes = nodes_
            i_jedge_dict = i_jedge_dict_

    return np.array(nodes)


def split_dataset(dataset, ratio):
    """Shuffle and split a dataset."""
    # np.random.seed(1234)  # fix the seed for shuffle.
    # np.random.shuffle(dataset)
    n = int(ratio * len(dataset))
    return dataset[:n], dataset[n:]


def create_datasets(task, data_for_gnn, radius, device):

    dir_dataset =  ''
    # print(type(data_for_gnn))
    print(data_for_gnn.shape)
    """Initialize x_dict, in which each key is a symbol type
    (e.g., atom and chemical bond) and each value is its index.
    """
    atom_dict = defaultdict(lambda: len(atom_dict))
    bond_dict = defaultdict(lambda: len(bond_dict))
    fingerprint_dict = defaultdict(lambda: len(fingerprint_dict))
    edge_dict = defaultdict(lambda: len(edge_dict))
   
    
    non_working_smiles_index = []
    def create_dataset(data):

#         print(filename)

#         """Load a dataset."""
#         with open(dir_dataset + filename, 'r') as f:
#             smiles_property = f.readline().strip().split()
#             data_original = f.read().strip().split('\n')

#         """Exclude the data contains '.' in its smiles."""
#         data_original = [data for data in data_original
#                          if '.' not in data.split()[0]]

        dataset = []
        final_index = []

        for data_index in range(len(data)):
#             print(data_index)
            try:
        #             smiles, property = data.strip().split()
                smiles1, smiles2, property = data.iloc[data_index]['drug_a_structure'], data.iloc[data_index]['drug_b_structure'], data.iloc[data_index]['synergy']

                """Create each data with the above defined functions."""
                # for drug A:
                mol = Chem.AddHs(Chem.MolFromSmiles(smiles1))
                atoms = create_atoms(mol, atom_dict)
                molecular_size1 = len(atoms)
                i_jbond_dict = create_ijbonddict(mol, bond_dict)
                fingerprints1 = extract_fingerprints(radius, atoms, i_jbond_dict,
                                                    fingerprint_dict, edge_dict)
                adjacency1 = Chem.GetAdjacencyMatrix(mol)

                # for drug B:
                mol = Chem.AddHs(Chem.MolFromSmiles(smiles2))
                atoms = create_atoms(mol, atom_dict)
                molecular_size2 = len(atoms)
                i_jbond_dict = create_ijbonddict(mol, bond_dict)
                fingerprints2 = extract_fingerprints(radius, atoms, i_jbond_dict,
                                                    fingerprint_dict, edge_dict)
                adjacency2 = Chem.GetAdjacencyMatrix(mol)

                """Transform the above each data of numpy
                to pytorch tensor on a device (i.e., CPU or GPU).
                """
                fingerprints1 = torch.LongTensor(fingerprints1).to(device)
                adjacency1 = torch.FloatTensor(adjacency1).to(device)
                fingerprints2 = torch.LongTensor(fingerprints2).to(device)
                adjacency2 = torch.FloatTensor(adjacency2).to(device)

                if task == 'regression':
                    property = torch.FloatTensor([[float(property)]]).to(device)

                dataset.append((fingerprints1, adjacency1, molecular_size1, fingerprints2, adjacency2, molecular_size2, property))
                final_index.append(data_index)
            
            except:
                non_working_smiles_index.append(data_index)
            
        return dataset, final_index

    dataset_train, final_index = create_dataset(data_for_gnn)
    print('Skipped '+ str(len(non_working_smiles_index)) + ' smiles in training data')
    dataset_train, dataset_dev = split_dataset(dataset_train, 0.9)
    # dataset_test = create_dataset(data_for_gnn[5000:])
    dataset_test = dataset_dev

    N_fingerprints = len(fingerprint_dict)

    return dataset_train, dataset_dev, dataset_test, N_fingerprints, final_index



import sys
import timeit

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import roc_auc_score

# import preprocess as pp


class MolecularGraphNeuralNetwork(nn.Module):
    def __init__(self, N_fingerprints, dim, layer_hidden, layer_output, nfeat, nhid, nclass, dropout):
        super(MolecularGraphNeuralNetwork, self).__init__()
        self.embed_fingerprint = nn.Embedding(N_fingerprints, dim)
        self.W_fingerprint = nn.ModuleList([nn.Linear(dim, dim)
                                            for _ in range(layer_hidden)])
        self.W_output = nn.ModuleList([nn.Linear(dim+dim+dim+dim, dim+dim+dim+dim)
                                       for _ in range(layer_output)])
        if task == 'classification':
            self.W_property = nn.Linear(dim, 2)
        if task == 'regression':
            self.W_property = nn.Linear(dim+dim+dim+dim, 1)
        
        # gcn
        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, 128)
        self.dropout = dropout
        self.output = nn.Linear(128, 100).float()

    def pad(self, matrices, pad_value):
        """Pad the list of matrices
        with a pad_value (e.g., 0) for batch processing.
        For example, given a list of matrices [A, B, C],
        we obtain a new matrix [A00, 0B0, 00C],
        where 0 is the zero (i.e., pad value) matrix.
        """
        shapes = [m.shape for m in matrices]
        M, N = sum([s[0] for s in shapes]), sum([s[1] for s in shapes])
        zeros = torch.FloatTensor(np.zeros((M, N))).to(device)
        pad_matrices = pad_value + zeros
        i, j = 0, 0
        for k, matrix in enumerate(matrices):
            m, n = shapes[k]
            pad_matrices[i:i+m, j:j+n] = matrix
            i += m
            j += n
        return pad_matrices

    def update(self, matrix, vectors, layer):
        hidden_vectors = torch.relu(self.W_fingerprint[layer](vectors))
        return hidden_vectors + torch.matmul(matrix, hidden_vectors)

    def sum(self, vectors, axis):
        sum_vectors = [torch.sum(v, 0) for v in torch.split(vectors, axis)]
        return torch.stack(sum_vectors)

    def mean(self, vectors, axis):
        mean_vectors = [torch.mean(v, 0) for v in torch.split(vectors, axis)]
        return torch.stack(mean_vectors)

    def gnn(self, inputs):

        """Cat or pad each input data for batch processing."""
        fingerprints, adjacencies, molecular_sizes = inputs
        fingerprints = torch.cat(fingerprints)
        adjacencies = self.pad(adjacencies, 0)

        """GNN layer (update the fingerprint vectors)."""
        fingerprint_vectors = self.embed_fingerprint(fingerprints)
        for l in range(layer_hidden):
            hs = self.update(adjacencies, fingerprint_vectors, l)
            fingerprint_vectors = F.normalize(hs, 2, 1)  # normalize.

        """Molecular vector by sum or mean of the fingerprint vectors."""
        molecular_vectors = self.sum(fingerprint_vectors, molecular_sizes)
        # molecular_vectors = self.mean(fingerprint_vectors, molecular_sizes)

        return molecular_vectors

    def mlp(self, vectors):
        """Classifier or regressor based on multilayer perceptron."""
        for l in range(layer_output):
#             print(self.W_output[l](vectors))
#             print(self.W_output[l](vectors).shape)
            vectors = torch.relu(self.W_output[l](vectors))
        outputs = self.W_property(vectors)
        return outputs

    def forward_classifier(self, data_batch, train):

        inputs = data_batch[:-1]
        correct_labels = torch.cat(data_batch[-1])

        if train:
            molecular_vectors = self.gnn(inputs)
            predicted_scores = self.mlp(molecular_vectors)
            loss = F.cross_entropy(predicted_scores, correct_labels)
            return loss
        else:
            with torch.no_grad():
                molecular_vectors = self.gnn(inputs)
                predicted_scores = self.mlp(molecular_vectors)
            predicted_scores = predicted_scores.to('cpu').data.numpy()
            predicted_scores = [s[1] for s in predicted_scores]
            correct_labels = correct_labels.to('cpu').data.numpy()
            return predicted_scores, correct_labels

    def forward_regressor(self, data_batch, x, adj, train):

        inputs_d1 = data_batch[:3]
        inputs_d2 = data_batch[3:-1]
        correct_values = torch.cat(data_batch[-1])
        
        if train:
            molecular_vectors_d1 = self.gnn(inputs_d1)
            molecular_vectors_d2 = self.gnn(inputs_d2)
#             molecular_vectors = np.concatenate([molecular_vectors_d1,molecular_vectors_d2])
            # print(molecular_vectors_d1.shape, molecular_vectors_d1.shape)
            x = F.relu(self.gc1(x, adj))
            x = F.dropout(x, self.dropout, training=self.training)
            x = self.gc2(x, adj)
            x = self.output(x.float())
            
            molecular_vectors = torch.cat((molecular_vectors_d1, molecular_vectors_d2, x), 1)
            # print(molecular_vectors.shape)
            predicted_values = self.mlp(molecular_vectors)
#             print(predicted_values.shape, correct_values.shape, molecular_vectors.shape, molecular_vectors_d1.shape)
            # print(predicted_values.shape)
            loss = F.mse_loss(predicted_values, correct_values)
            return loss
        # else:
        #     with torch.no_grad():
        #         molecular_vectors_d1 = self.gnn(inputs_d1)
        #         molecular_vectors_d2 = self.gnn(inputs_d2)
        #         molecular_vectors = torch.cat((molecular_vectors_d1, molecular_vectors_d2), 1)
        #         predicted_values = self.mlp(molecular_vectors)
        #     predicted_values = predicted_values.to('cpu').data.numpy()
        #     correct_values = correct_values.to('cpu').data.numpy()
        #     predicted_values = np.concatenate(predicted_values)
        #     correct_values = np.concatenate(correct_values)
        #     return predicted_values, correct_values


class Trainer(object):
    def __init__(self, model):
        self.model = model
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)

    def train(self, dataset, x, adj, train = True):
        np.random.shuffle(dataset)
        N = len(dataset)
        loss_total = 0
        data_batch = list(zip(*dataset[:]))
        if task == 'regression':
          loss = self.model.forward_regressor(data_batch, x, adj, train=True)
          self.optimizer.zero_grad()
          loss.backward()
          self.optimizer.step()
          loss_total += loss.item()
        # for i in range(0, N, batch_train):
        #     data_batch = list(zip(*dataset[i:i+batch_train]))
        #     if task == 'classification':
        #         loss = self.model.forward_classifier(data_batch, train=True)
        #     if task == 'regression':
        #         loss = self.model.forward_regressor(data_batch, train=True)
        #     self.optimizer.zero_grad()
        #     loss.backward()
        #     self.optimizer.step()
        #     loss_total += loss.item()
        return loss_total


class Tester(object):
    def __init__(self, model):
        self.model = model

    def test_regressor(self, dataset):
        N = len(dataset)
        SAE = 0  # sum absolute error.
        for i in range(0, N, batch_test):
            data_batch = list(zip(*dataset[i:i+batch_test]))
            predicted_values, correct_values = self.model.forward_regressor(
                                               data_batch, train=False)
            SAE += sum(np.abs(predicted_values-correct_values))
        MAE = SAE / N  # mean absolute error.
        return MAE

    def save_result(self, result, filename):
        with open(filename, 'a') as f:
            f.write(result + '\n')





In [ ]:
import math

import torch

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module


class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input.double(), self.weight.double())
        adj = torch.from_numpy(adj).double()
#         print(type(adj), type(support))
        # print(adj.shape, support.shape)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, 128)
        self.dropout = dropout
        # self.output = nn.Linear(128, nclass).float()
        self.output = nn.Linear(128, 100).float()

    def forward(self, x, adj):
#         x = x.astype('double')
#         adj = adj.astype('double')
        
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        x = self.output(x.float())
#         print(x)
        
        return x

In [ ]:
# features_train.shape
# # Create dummy adjacency matrix for now
# import scipy.sparse as sp
# adj = sp.coo_matrix((len(data_for_gnn_gcn[:100]), len(data_for_gnn_gcn[:100])), dtype=np.float32).toarray()

# # build symmetric adjacency matrix
# adj = adj + np.matmul(adj.T, adj.T > adj) - np.matmul(adj, adj.T > adj)
# adj
# adj.shape


In [ ]:
# # calculate adj
# import scipy.sparse as sp
# adj = sp.coo_matrix((100, 100), dtype=np.float32).toarray()
# # build symmetric adjacency matrix
# adj = adj + np.matmul(adj.T, adj.T > adj) - np.matmul(adj, adj.T > adj)

In [ ]:
def get_gcn_dataset(adj):
  data_for_gcn = data_for_gnn_gcn['cell_line_expr']
  gcn_train_data, gcn_dev_data = split_dataset(data_for_gcn, 0.9)
  # gcn_train_data = gcn_train_data
  # gcn_dev_data = gcn_dev_data
  gcn_test_data = gcn_dev_data
  # print(gcn_train_data.shape, gcn_dev_data.shape)

  import scipy.sparse as sp
  features_all = pd.DataFrame(gcn_train_data.to_list()).values
  features_train = sp.csr_matrix(features_all, dtype=np.float32)
  features_train = normalize(features_train)
  features_train = torch.FloatTensor(np.array(features_train.todense()))

  features_all = pd.DataFrame(gcn_dev_data.to_list()).values
  features_dev = sp.csr_matrix(features_all, dtype=np.float32)
  features_dev = normalize(features_dev)
  features_dev = torch.FloatTensor(np.array(features_dev.todense()))

  features_all = pd.DataFrame(gcn_test_data.to_list()).values
  features_test = sp.csr_matrix(features_all, dtype=np.float32)
  features_test = normalize(features_test)
  features_test = torch.FloatTensor(np.array(features_test.todense()))

  print('gcn input train, dev, test size: ', features_train.shape, features_dev.shape, features_test.shape )
  
  adj = normalize(adj + sp.eye(adj.shape[0]))

  
  labels = torch.LongTensor(data_for_gnn_gcn['synergy'])
  adj1 = sparse_mx_to_torch_sparse_tensor(sp.csr_matrix(adj))

  idx_train = gcn_train_data.index
  idx_val = gcn_dev_data.index
  idx_test = gcn_test_data.index

  idx_train = torch.LongTensor(idx_train)
  idx_val = torch.LongTensor(idx_val)
  idx_test = torch.LongTensor(idx_test)

  return features_train, features_dev, features_test, idx_train, idx_val, idx_test, adj


In [ ]:

# Run the main code function: 
task = 'regression'
dataset = ''
radius = 3
dim = 50
layer_hidden=6
layer_output=6

batch_train=32
batch_test=32
lr=1e-4
lr_decay=0.99
decay_interval=10
iteration=5

#     (task, dataset, radius, dim, layer_hidden, layer_output,
#      batch_train, batch_test, lr, lr_decay, decay_interval, iteration,
#      setting) = sys.argv[1:]
(radius, dim, layer_hidden, layer_output,
  batch_train, batch_test, decay_interval,
  iteration) = map(int, [radius, dim, layer_hidden, layer_output,
                        batch_train, batch_test,
                        decay_interval, iteration])
lr, lr_decay = map(float, [lr, lr_decay])
#     device = torch.device('cuda')
# device = torch.device('cpu')

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('The code uses a GPU!')
else:
    device = torch.device('cpu')
    print('The code uses a CPU...')
#     print('-'*100)

print('Preprocessing the GNN', dataset, 'dataset.')
print('Just a moment......')
(dataset_train, dataset_dev, dataset_test,
  N_fingerprints, final_index) = create_datasets(task, data_for_gnn_gcn, radius, device)
print('-'*100)

# dataset_train = dataset_train[:100]
# dataset_dev = dataset_dev[-500:]

print('The preprocess has finished!')
print('# of training data samples:', len(dataset_train))
print('# of development data samples:', len(dataset_dev))
print('# of test data samples:', len(dataset_test))
print('-'*100)


In [ ]:
# Remove the index which were skipped
data_for_gnn_gcn = data_for_gnn_gcn.iloc[final_index].reset_index()


In [ ]:
print('Preprocessing the GCN dataset.')
print('Just a moment......')
features_train, features_dev, features_test, idx_train, idx_val, idx_test, adj = get_gcn_dataset(adj)
print('-'*100)

In [ ]:

################################################################## RUN MODELLL

dim = 50
iteration=50
print('Creating a model.')
torch.manual_seed(1234)


model = MolecularGraphNeuralNetwork(
        N_fingerprints, dim, layer_hidden, layer_output, nfeat=features_train.shape[1],nhid=16,nclass=1,dropout=0.2 ).to(device)
trainer = Trainer(model)
# tester = Tester(model)
print('# of model parameters:',
      sum([np.prod(p.size()) for p in model.parameters()]))
print('-'*100)

file_result = train_data_path + '/setting1_run.txt'

if task == 'regression':
    result = 'Epoch\tTime(sec)\tLoss_train\tMAE_dev\tMAE_test'

with open(file_result, 'w') as f:
    f.write(result + '\n')

print('Start training.')
print('The result is saved in the output directory every epoch!')

all_loss_train = []


# np.random.seed(1234)

start = timeit.default_timer()

# BATCH THIS HERE AND ADJ MATRIX CALC EVERY TIME

# epoch_batch_size = int( len(dataset_train)/iteration )
epoch_batch_size = 100
batch_start_nd = 0
batch_end_nd = epoch_batch_size
iteration = int(len(dataset_train)/epoch_batch_size)
print(iteration)

for epoch in range(iteration*2):

    # Dont decay lr
    # if epoch % decay_interval == 0:
    #     trainer.optimizer.param_groups[0]['lr'] *= lr_decay

    ds_train = dataset_train[batch_start_nd:batch_end_nd]
    fs_train = features_train[batch_start_nd:batch_end_nd]


    print(epoch)
    loss_train = trainer.train(ds_train, fs_train, adj, train=True)
    print('done')
    # if task == 'regression':
    #     prediction_dev = tester.test_regressor(dataset_dev)
    #     prediction_test = tester.test_regressor(dataset_test)
    prediction_test,prediction_dev = 0, 0

    time = timeit.default_timer() - start

    if epoch == 1:
        minutes = time * iteration / 60
        hours = int(minutes / 60)
        minutes = int(minutes - 60 * hours)
        print('The training will finish in about',
              hours, 'hours', minutes, 'minutes.')
        print('-'*100)
        print(result)

    result = '\t'.join(map(str, [epoch, time, loss_train,
                                 prediction_dev, prediction_test]))
    # tester.save_result(result, file_result)

    print(result)
    all_loss_train.append(loss_train)
    epoch += 1

    batch_start_nd = batch_end_nd
    batch_end_nd = batch_start_nd + epoch_batch_size
    if(epoch == iteration):
      batch_start_nd = 0
      batch_end_nd = epoch_batch_size

Creating a model.
# of model parameters: 519593
----------------------------------------------------------------------------------------------------
Start training.
The result is saved in the output directory every epoch!
79
0
done
0	2.9097195320000537	513.2335815429688	0	0
1
done
The training will finish in about 0 hours 7 minutes.
----------------------------------------------------------------------------------------------------
0	2.9097195320000537	513.2335815429688	0	0
1	5.4772163469997395	398.6176452636719	0	0
2
done
2	8.019814188000055	321.8485107421875	0	0
3
done
3	10.437578408999798	319.814697265625	0	0
4
done
4	12.863119435999579	752.2893676757812	0	0
5
done
5	15.368189077999887	563.160888671875	0	0
6
done
6	17.80202894499962	701.803955078125	0	0
7
done
7	20.70601590899969	451.8525085449219	0	0
8
done
8	23.1158027219999	620.16455078125	0	0
9
done
9	25.285636264999994	456.2766418457031	0	0
10
done
10	27.925222621000103	853.5056762695312	0	0
11
done
11	30.817600622999635	681.81

The code uses a CPU...
Preprocessing the GNN  dataset.
Just a moment......
(10000, 5)
Skipped 1131 smiles in training data
----------------------------------------------------------------------------------------------------
The preprocess has finished!
# of training data samples: 7982
# of development data samples: 887
# of test data samples: 887
----------------------------------------------------------------------------------------------------


Preprocessing the GCN dataset.
Just a moment......
gcn input train, dev, test size:  torch.Size([7982, 10000]) torch.Size([887, 10000]) torch.Size([887, 10000])
----------------------------------------------------------------------------------------------------
gcn input train, dev, test size:  torch.Size([7982, 10000]) torch.Size([887, 10000]) torch.Size([887, 10000])


In [ ]:
n = int(0.9 * 5000)
n

4500

In [ ]:

################################################################## RUN MODELLL

dim = 50
iteration=100
print('Creating a model.')
torch.manual_seed(1234)
model = MolecularGraphNeuralNetwork(
        N_fingerprints, dim, layer_hidden, layer_output, ).to(device)
trainer = Trainer(model)
tester = Tester(model)
print('# of model parameters:',
      sum([np.prod(p.size()) for p in model.parameters()]))
print('-'*100)

file_result = train_data_path + '/setting1_run.txt'

if task == 'regression':
    result = 'Epoch\tTime(sec)\tLoss_train\tMAE_dev\tMAE_test'

with open(file_result, 'w') as f:
    f.write(result + '\n')

print('Start training.')
print('The result is saved in the output directory every epoch!')

all_loss_train = []


# np.random.seed(1234)

start = timeit.default_timer()

for epoch in range(iteration):

    # Dont decay lr
    # if epoch % decay_interval == 0:
    #     trainer.optimizer.param_groups[0]['lr'] *= lr_decay

    loss_train = trainer.train(dataset_train)

    if task == 'regression':
        prediction_dev = tester.test_regressor(dataset_dev)
        prediction_test = tester.test_regressor(dataset_test)

    time = timeit.default_timer() - start

    if epoch == 1:
        minutes = time * iteration / 60
        hours = int(minutes / 60)
        minutes = int(minutes - 60 * hours)
        print('The training will finish in about',
              hours, 'hours', minutes, 'minutes.')
        print('-'*100)
        print(result)

    result = '\t'.join(map(str, [epoch, time, loss_train,
                                 prediction_dev, prediction_test]))
    tester.save_result(result, file_result)

    print(result)
    all_loss_train.append(loss_train)
    epoch += 1


In [ ]:
!nvidia-smi

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()